In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
caminho_arquivo = '/content/drive/My Drive/Dataset/dataset.csv'

dataframe = pd.read_csv(caminho_arquivo)
dataframe.head()

,Nome da Música,Artista,Gênero Musical,Letra da Música
0,Carolina,Seu Jorge,MPB,Carolina é uma menina bem difícil de esquecer ...
1,Epitáfio,Titãs,Rock,Devia ter amado mais Ter chorado mais Ter vist...
2,Lugar Ao Sol,Charlie Brown Jr.,Rock,"Que bom viver, como é bom sonhar E o que ficou..."
3,Relicário - Ao Vivo,Cássia Eller,Rock,É uma índia com colar A tarde linda que não qu...
4,Você Me Vira A Cabeça (Me Tira Do Sério),Alcione,Samba,"Você me vira a cabeça, me tira do sério Destró..."


In [ ]:
dataframe.shape

(146612, 4)

In [ ]:
import re

def limpar_texto(musica):
    if isinstance(musica, str):
        musica = re.sub(r'\(.*?\)', '', musica)  # Remove tudo que está entre parênteses
        musica = re.sub(r'\[.*?\]', '', musica)  # Remove tudo que está entre colchetes
        musica = re.sub(r'\n+', ' ', musica)  # Remove os caracteres "\n"
        musica = re.sub(r'\s+', ' ', musica).strip()  # Remove espaços extras
    return musica

dataframe['Letra da Música'] = dataframe['Letra da Música'].apply(limpar_texto)

In [ ]:
import spacy

!python -m spacy download pt_core_news_lg
pln = spacy.load('pt_core_news_lg')
pln

from spacy.matcher import DependencyMatcher

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 1.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
def carregar_palavras(nome_arquivo):
    with open(nome_arquivo, 'r') as arquivo:
        conteudo = arquivo.read()
        palavras = conteudo.split()
        return palavras

nome_arquivo = '/content/drive/MyDrive/TCC - UFMS/BRACIS/Profissões/Lista de Profissões/profissoes_fem.txt'
palavras_procuradas = carregar_palavras(nome_arquivo)
print(palavras_procuradas)

nome_arquivo = '/content/drive/MyDrive/TCC - UFMS/BRACIS/Profissões/Lista de Profissões/profissoes_casos_especiais.txt'
palavras_neutras = carregar_palavras(nome_arquivo)
print(palavras_neutras)

profissoes = palavras_procuradas + palavras_neutras

['legisladora', 'ministra', 'diretora', 'professora', 'engenheira', 'desenvolvedora', 'programadora', 'agrônoma', 'arquiteta', 'bióloga', 'médica', 'veterinária', 'farmacêutica', 'enfermeira', 'técnica', 'advogada', 'procuradora', 'governadora', 'vereadora', 'prefeita', 'deputada', 'senadora', 'defensora', 'juíza', 'promotora', 'delegada', 'psicóloga', 'pesquisadora', 'historiadora', 'filósofa', 'auditora', 'administradora', 'contadora', 'secretária', 'redatora', 'tradutora', 'escritora', 'locutora', 'produtora', 'coreógrafa', 'atriz', 'compositora', 'decoradora', 'bailarina', 'cantora', 'escultora', 'pintora', 'floricultora', 'inspetora', 'corretora', 'leiloeira', 'avaliadora', 'bibliotecária', 'fotógrafa', 'apresentadora', 'dançarina', 'escrivã', 'entrevistadora', 'governanta', 'cozinheira', 'camareira', 'garçonete', 'copeira', 'astróloga', 'bombeira', 'lavadeira', 'repositora', 'instaladora', 'pescadora', 'caçadora', 'montadora', 'joalheira', 'vidreira', 'artesã', 'marceneira', 'car

In [ ]:
sujeitos_femininos = ["mulher", "mulheres", "ela", "elas", "menina", "meninas", "garota", "garotas", "senhora", "senhoras", "senhorita", "senhoritas", "moça", "moças", "donzela", "donzelas", "dama", "damas", "rainha", "rainhas", "princesa", "esposa", "esposas", "namorada", "namoradas", "novinha", "novinhas", "mina", "minas"]
sujeitos_masculinos = ["homem", "homens", "ele", "eles", "menino", "meninos", "garoto", "garotos", "senhor", "senhores", "rapaz", "rapazes", "moço", "moços", "cavalheiro", "cavalheiros", "rei", "reis", "príncipe",  "marido", "maridos", "namorado", "namorados", "novinho", "novinhos"]

#Extraindo Profissões no Feminino - ARVORE DE DEPENDENCIA - ajustado para profissoes

In [ ]:
dep_matcher = DependencyMatcher(vocab=pln.vocab)


#ela é professora
#professora pode ser: adj ou subst
sujeitoauxprofissao = [{"RIGHT_ID": "profissao", "RIGHT_ATTRS": {"LOWER": {"IN": profissoes}}},
                      {"LEFT_ID": "profissao", "REL_OP": ">", "RIGHT_ID": "sujeito", "RIGHT_ATTRS": {"DEP": "nsubj", "LOWER": {"IN": sujeitos_femininos}}},
                      {"LEFT_ID": "profissao", "REL_OP": ">", "RIGHT_ID": "auxiliar", "RIGHT_ATTRS":  {"POS":{"IN":["AUX", "COP"]}}}]

parecer_tornar = [{"RIGHT_ID": "verbo", "RIGHT_ATTRS": {"LEMMA":
                        {"IN": ["parecer", "tornar"]}}},
                        {"LEFT_ID": "verbo", "REL_OP": ">", "RIGHT_ID": "sujeito", "RIGHT_ATTRS": {"DEP": "nsubj", "LOWER": {"IN": sujeitos_femininos}}},
                        {"LEFT_ID": "verbo", "REL_OP":">", "RIGHT_ID": "profissao", "RIGHT_ATTRS": {"LOWER": {"IN": profissoes}}}]

#no você busca apenas as profissoes onde é o gênero explicito 'você é professora'; diante da dificuldade de verificar gênero de 'você é piloto'
voceauxprofissao = [{"RIGHT_ID": "profissao", "RIGHT_ATTRS": {"LOWER": {"IN": palavras_procuradas}}},
                   {"LEFT_ID": "profissao", "REL_OP": ">", "RIGHT_ID": "sujeito", "RIGHT_ATTRS": {"DEP": "nsubj", "LOWER": {"in": ["você", "vocês"]}}},
                   {"LEFT_ID": "profissao", "REL_OP": ">", "RIGHT_ID": "auxiliar", "RIGHT_ATTRS": {"POS":{"IN":["AUX", "COP"]}}}]


voce_parecer_tornar = [{"RIGHT_ID": "verbo", "RIGHT_ATTRS": {"LEMMA":
                             {"IN": ["parecer",  "tornar"]}}},
                             {"LEFT_ID": "verbo", "REL_OP": ">", "RIGHT_ID": "sujeito", "RIGHT_ATTRS": {"DEP": "nsubj", "LOWER": {"IN": ["você", "vocês"]}}},
                             {"LEFT_ID": "verbo", "REL_OP":">", "RIGHT_ID": "adjetivo", "RIGHT_ATTRS": {"LOWER": {"IN": palavras_procuradas}}}]


# substantivo = [{"RIGHT_ID": "substantivo", "RIGHT_ATTRS": {"POS": "NOUN"}},
#                {"LEFT_ID": "substantivo", "REL_OP": ">", "RIGHT_ID": "sujeito", "RIGHT_ATTRS": {"DEP": "nsubj", "LOWER": {"IN": sujeitos_femininos}}},
#                {"LEFT_ID": "substantivo", "REL_OP": ">", "RIGHT_ID": "auxiliar", "RIGHT_ATTRS": {"POS":{"IN":["AUX", "COP"]}}}]

sujeitoauxprofissao_nomeproprio = [{"RIGHT_ID": "profissao",  "RIGHT_ATTRS":{"LOWER": {"IN": profissoes}}},
                      {"LEFT_ID": "profissao", "REL_OP": ">", "RIGHT_ID": "sujeito", "RIGHT_ATTRS": {"DEP": "nsubj", "ENT_TYPE": "PER",'POS': 'PROPN', "MORPH": {"IN": ['Gender=Fem|Number=Sing']}}},
                      {"LEFT_ID": "profissao", "REL_OP": ">", "RIGHT_ID": "auxiliar", "RIGHT_ATTRS": {"POS":{"IN":["AUX", "COP"]}}}]

parecer_tornar_nomeproprio = [{"RIGHT_ID": "verbo", "RIGHT_ATTRS": {"LEMMA":
                          {"IN": ["parecer", "tornar"]}}},
                        {"LEFT_ID": "verbo", "REL_OP": ">", "RIGHT_ID": "sujeito", "RIGHT_ATTRS": {"DEP": "nsubj", "ENT_TYPE": "PER",'POS': 'PROPN', "MORPH": {"IN": ['Gender=Fem|Number=Sing']}}},
                        {"LEFT_ID": "verbo", "REL_OP":">", "RIGHT_ID": "profissao", "RIGHT_ATTRS": {"LOWER": {"IN": profissoes}}}]

# sujeitoprofissao_nomeproprio = [{"RIGHT_ID": "noun", "RIGHT_ATTRS": {"ENT_TYPE": "PER",'POS': 'PROPN', "MORPH": {"IN": ['Gender=Fem|Number=Sing']}}},
#                    {"LEFT_ID": "noun", "REL_OP":">", "RIGHT_ID": "profissao", "RIGHT_ATTRS": {"LOWER": {"IN": profissoes}}}]

# substantivo_nomeproprio = [{"RIGHT_ID": "substantivo", "RIGHT_ATTRS": {"POS": "NOUN"}},
#                {"LEFT_ID": "substantivo", "REL_OP": ">", "RIGHT_ID": "sujeito", "RIGHT_ATTRS": {"DEP": "nsubj", "ENT_TYPE": "PER",'POS': 'PROPN', "MORPH": {"IN": ['Gender=Fem|Number=Sing']}}},
#                {"LEFT_ID": "substantivo", "REL_OP": ">", "RIGHT_ID": "auxiliar", "RIGHT_ATTRS": {"POS":{"IN":["AUX", "COP"]}}}]


# dep_matcher.add('sujeitoprofissao_nomeproprio', patterns=[sujeitoprofissao_nomeproprio])
dep_matcher.add('parecer_tornar', patterns=[parecer_tornar])
dep_matcher.add('sujeitoauxprofissao', patterns=[sujeitoauxprofissao])
dep_matcher.add('voceauxprofissao', patterns=[voceauxprofissao])
dep_matcher.add('voce_parecer_tornar', patterns=[voce_parecer_tornar])
dep_matcher.add('sujeitoauxprofissao_nomeproprio', patterns=[sujeitoauxprofissao_nomeproprio])
dep_matcher.add('parecer_tornar_nomeproprio', patterns=[parecer_tornar_nomeproprio])


In [ ]:
resultados = []

for index, row in dataframe.iterrows():
    text = str(row['Letra da Música'])
    doc = pln(text)
    dep_matches = dep_matcher(doc)
    for match in dep_matches:
        pattern_name = match[0]
        matches = match[1]
        if pln.vocab[pattern_name].text in ['voceauxprofissao', 'sujeitoauxprofissao','sujeitoauxprofissao_nomeproprio'] :
            inicio, final = matches[1], matches[0]
            frase = doc[inicio:final+1].text
            resultados.append({'Padrão': pln.vocab[pattern_name].text,
                           'Frase': frase,
                           'Nome da Música': row['Nome da Música'],
                           'Gênero Musical': row['Gênero Musical'],
                           'Letra da Música': text})
        elif pln.vocab[pattern_name].text in ['parecer_tornar', 'voce_parecer_tornar','parecer_tornar_nomeproprio']:
            inicio, final = matches[1], matches[2]
            frase = doc[inicio:final+1].text
            resultados.append({'Padrão': pln.vocab[pattern_name].text,
                           'Frase': frase,
                           'Nome da Música': row['Nome da Música'],
                           'Gênero Musical': row['Gênero Musical'],
                           'Letra da Música': text})
        # elif pln.vocab[pattern_name].text == 'sujeitoprofissao_nomeproprio':
        #     inicio, final = matches[0], matches[1]
        #     frase = doc[inicio:final+1].text
        #     resultados.append({'Padrão': pln.vocab[pattern_name].text,
        #                    'Frase': frase,
        #                    'Nome da Música': row['Nome da Música'],
        #                    'Gênero Musical': row['Gênero Musical'],
        #                    'Letra da Música': text
        #                    })

resultados_df = pd.DataFrame(resultados)
resultados_df.head()

,Padrão,Frase,Nome da Música,Gênero Musical,Letra da Música
0,sujeitoauxprofissao,elas são atriz,Ballena,Rap,"Duas doses, bebida rosa De salto alto, prada m..."
1,sujeitoauxprofissao_nomeproprio,Sandrinha era caçadora,Sandrinha,pop,Sandrinha Sandrinha era a melhor garota da pra...
2,voceauxprofissao,você foi professora,Palco do Mundo,sertanejo,Palco do Mundo Mulher você foi professora na m...
3,sujeitoauxprofissao,Mulher você foi professora,Palco Do Mundo,sertanejo,Mulher você foi professora na minha jornada Na...
4,voceauxprofissao,você foi professora,Palco Do Mundo,sertanejo,Mulher você foi professora na minha jornada Na...


In [ ]:
resultados_df['Frase'] = resultados_df['Frase'].astype(str)
resultados_df['Frase'] = resultados_df['Frase'].str.strip()
resultados_df['Frase'] = resultados_df['Frase'].str.lower()
resultados_df['Frase'] = resultados_df['Frase'].str.replace(r'\s+', ' ', regex=True)
resultados_dfs = resultados_df.drop_duplicates(subset=['Frase', 'Letra da Música'])
resultados_df.head()
resultados_dfs.head()

,Padrão,Frase,Nome da Música,Gênero Musical,Letra da Música
0,sujeitoauxprofissao,elas são atriz,Ballena,Rap,"Duas doses, bebida rosa De salto alto, prada m..."
1,sujeitoauxprofissao_nomeproprio,sandrinha era caçadora,Sandrinha,pop,Sandrinha Sandrinha era a melhor garota da pra...
2,voceauxprofissao,você foi professora,Palco do Mundo,sertanejo,Palco do Mundo Mulher você foi professora na m...
3,sujeitoauxprofissao,mulher você foi professora,Palco Do Mundo,sertanejo,Mulher você foi professora na minha jornada Na...
4,voceauxprofissao,você foi professora,Palco Do Mundo,sertanejo,Mulher você foi professora na minha jornada Na...


In [ ]:
resultados_df.shape

(91, 5)

In [ ]:
from google.colab import files
resultados_df.to_csv('arvore_fem.csv', index=False)
files.download('arvore_fem.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>